In [ ]:
import os
import time
import glob
import json
import torch
import pickle
import random

import numpy as np
import matplotlib.pyplot as plt

from BERT import *
from data import *
from utils.remi import *
from utils.utils import *
from utils.vocab import *

%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
### load config file
with open("./config.json", "r") as f:
    config = json.load(f)

### fix random seed
random_seed = config["random_seed"]

# it may slow computing performance
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

random.seed(random_seed)
np.random.seed(random_seed)

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

#### initialize model with GPU
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
### define model
model = BERT_Lightning(
    dim=config["dim"],
    depth=config["depth"],
    heads=config["heads"],
    dim_head=int(config["dim"] / config["heads"]),
    mlp_dim=int(4 * config["dim"]),
    max_len=config["max_len"],
    rate=config["rate"],
    loss_weights=config["loss_weights"],
    lr=config["lr"],
    warm_up=config["warm_up"],
    temp=config["temp"],
    mode=config["mode"],
)

In [ ]:
folder_path = "/workspace/loop_season_2/model"
model_path = "BERT-random_seed_0-batch_size_24-num_workers_16-dim_768-depth_12-heads_12-max_len_512-rate_0-masking_0.8-replace_0.1-loss_weights_[1, 0.1]-lr_0.0001-epochs_3-warm_up_10000-temp_0.1-gpus_[2, 3, 4, 5, 6, 7]-mode_BERT-neighbor-epoch=2-val_loss=0.7468.ckpt"

model = model.load_from_checkpoint(
    os.path.join(folder_path, model_path),
    dim=config["dim"],
    depth=config["depth"],
    heads=config["heads"],
    dim_head=int(config["dim"] / config["heads"]),
    mlp_dim=int(4 * config["dim"]),
    max_len=config["max_len"],
    rate=config["rate"],
    loss_weights=config["loss_weights"],
    lr=config["lr"],
    warm_up=config["warm_up"],
    temp=config["temp"],
    mode=config["mode"],
)

model = model.to(device)
mode = model_path.split("mode_")[1].split("-epoch")[0]

#### Linear Probing

In [ ]:
folder_path = "./data/lmd_full_remi/"
folder_list = glob.glob(os.path.join(folder_path, "*"))

train_folder, val_folder, test_folder = dataset_split(folder_list)

task = {
    "inst": "multi-label",
    "chord": "multi-label",
    "tempo": "multi-class",
    "mean_velocity": "regression",
    "mean_duration": "regression",
    "groove_pattern": "multi-label",
}

metrics = {}
for i in range(5):
    start_time = time.time()

    train_files = folder_to_multiple_file(train_folder, k=5)
    test_files = folder_to_multiple_file(test_folder, k=10)

    random.shuffle(train_files)

    ### load dataloader
    train_module = DataModule(
        train_files,
        batch_size=config["batch_size"] * 8,
        num_workers=config["num_workers"],
        masking=config["masking"],
        replace=config["replace"],
        phase="train",
    )

    test_module = DataModule(
        test_files,
        batch_size=config["batch_size"] * 8,
        num_workers=config["num_workers"],
        phase="test",
    )

    train_set = train_module.return_dataloader()
    test_set = test_module.return_dataloader()

    train_feat, train_labels = get_feat(train_set, model, device)
    test_feat, test_labels = get_feat(test_set, model, device)

    metrics = probing(
        (np.vstack(train_feat[11]), train_labels),
        (np.vstack(test_feat[11]), test_labels),
        metrics,
        task,
    )

    metrics = clustering(
        (np.vstack(train_feat[11]), train_labels["file_name"]),
        (np.vstack(test_feat[11]), test_labels["file_name"]),
        metrics,
        num_k=1000,
    )

    print(f"{i+1} iter : {time.time() - start_time:.3f} sec")

In [ ]:
# save
with open(os.path.join("./results/", mode + "_results.pkl"), "wb") as f:
    pickle.dump(metrics, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# get mean and std of metrics
metrics_mean_std = dict(zip(metrics.keys(), map(lambda x: (np.mean(x), np.std(x)), metrics.values())))

print(metrics_mean_std)

#### Linear Probing for each layer

In [ ]:
metrics = {}
for i in range(12):
    start_time = time.time()

    metrics = probing(
        (np.vstack(train_feat[i]), train_labels),
        (np.vstack(test_feat[i]), test_labels),
        metrics,
        task,
    )

    metrics = clustering(
        (np.vstack(train_feat[i]), train_labels["file_name"]),
        (np.vstack(test_feat[i]), test_labels["file_name"]),
        metrics,
        num_k=1000,
    )

    print(f"{i+1} layer : {time.time() - start_time:.3f} sec")

In [ ]:
# save
with open(os.path.join("./results/", mode + "_results_each_layer.pkl"), "wb") as f:
    pickle.dump(metrics, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
task = {
    "inst": "max",
    "chord": "max",
    "tempo": "max",
    "mean_velocity": "min",
    "mean_duration": "min",
    "groove_pattern": "max",
    "song_clustering": "min",
}

for k, v in metrics.items():
    if task[k] == "max":
        layer = np.argmax(v) + 1
        value = np.max(v)
    elif task[k] == "min":
        layer = np.argmin(v) + 1
        value = np.min(v)
    else:
        raise

    print(f"{k}:, {layer, value}")

In [ ]:
fig_1_label = ["inst", "chord", "tempo", "groove_pattern", "song_clustering"]
fig_2_label = ["mean_velocity", "mean_duration"]

FIG_SIZE = (26, 4)
NUM_FONT_SIZE = 16
CHAR_FONT_SIZE = 18
LABEL_PAD = 5

TICKS = np.arange(0, 12)
SAVE_PATH = "./images/"

plt.figure(figsize=FIG_SIZE)
plt.subplot(1, 2, 1)
plt.text(-4.8, 0.45, mode, fontsize=CHAR_FONT_SIZE + 6)
for label in fig_1_label:
    plt.plot(metrics[label], "-o")
plt.xticks(TICKS, labels=TICKS + 1, fontsize=NUM_FONT_SIZE)
plt.yticks(fontsize=NUM_FONT_SIZE)
plt.xlabel("BERT Layer", fontsize=CHAR_FONT_SIZE, labelpad=LABEL_PAD)
plt.ylabel("Performance", fontsize=CHAR_FONT_SIZE, labelpad=LABEL_PAD)
plt.legend(fig_1_label, fontsize=CHAR_FONT_SIZE - 4)
plt.grid()

plt.subplot(1, 2, 2)
for label in fig_2_label:
    plt.plot(metrics[label], "-o")
plt.xticks(TICKS, labels=TICKS + 1, fontsize=NUM_FONT_SIZE)
plt.yticks(fontsize=NUM_FONT_SIZE)
plt.xlabel("BERT Layer", fontsize=CHAR_FONT_SIZE, labelpad=LABEL_PAD)
plt.ylabel("Performance", fontsize=CHAR_FONT_SIZE, labelpad=LABEL_PAD)
plt.legend(fig_2_label, fontsize=CHAR_FONT_SIZE - 4)
plt.grid()

plt.tight_layout(w_pad=5.2)

path = os.path.join(SAVE_PATH, mode + ".pdf")
plt.savefig(path, dpi=1000, bbox_inches="tight", format="pdf", pad_inches=0)
plt.show()

In [ ]:
from sklearn.manifold import TSNE

x = np.vstack(test_feat[0])
x_embed = TSNE(n_components=2).fit_transform(x)

In [ ]:
file_list = []
for batch_idx, batch in enumerate(val_set):
    file_list.append(batch["file_name"])

file_list = [elem for sublist in file_list for elem in sublist]

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(x_embed[:, 0], x_embed[:, 1])
plt.show()